In [1]:
# Convert the csv of the translated narratives to an xml tree

from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import torch
import sys
from lxml import etree
import argparse
import calendar
import csv
import re
import subprocess
import time
import math 

In [2]:
# -*- coding: UTF-8 -*-
data={}
seq_length = 0
all_categories = []
tree = etree.parse('../data/mds+rct/train_adult_cat.xml')
for e in tree.iter("cghr_cat"):
        if e.text not in data:
             data[e.text]=[]
             all_categories.append(e.text)
for e in tree.iter("hindi_narrative","cghr_cat"):
    if e.tag == "hindi_narrative":
        value= u''.join(e.text)
        if(seq_length) < len(value):
            seq_length = len(value)
        
    if e.tag == 'cghr_cat':
        data[e.text].append(value)


# for k,v in data.iteritems():
#     print (k)
#     print ((u"\n".join(v)))

n_categories= len(all_categories)


In [3]:
all_text = ''
for v in data.itervalues():
    all_text = all_text + u"-".join(v)

vocab = set(all_text)
n_letters = len(vocab)

def letterToIndex(letter):
    return list(vocab).index(letter)

def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(narrative):
    tensor = torch.zeros(seq_length,1)
    for li, letter in enumerate(narrative):
        tensor[li][0] = letterToIndex(letter)
    for i in range (1,seq_length-len(narrative)):
        tensor[len(narrative)+i][0]=0
    return tensor

narr = data['1'][0]



In [12]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self,sequence_length,input_size, hidden_size, output_size):
        super(CNN, self).__init__()

        self.hidden_size = hidden_size
        self.seq_length = sequence_length
        self.encoder = nn.Embedding(input_size,20)
        self.conv2 = nn.Sequential(nn.Conv2d(1,3,kernel_size = (2,20),stride=1), nn.ReLU())
        self.conv3 = nn.Sequential(nn.Conv2d(1,3,kernel_size=(3,20),stride=1),nn.ReLU())
        self.conv4 = nn.Sequential(nn.Conv2d(1,3,kernel_size=(4,20),stride=1),nn.ReLU())
        self.conv5 = nn.Sequential(nn.Conv2d(1,3,kernel_size=(5,20),stride=1),nn.ReLU())
        self.conv6 = nn.Sequential(nn.Conv2d(1,3,kernel_size=(6,20),stride=1),nn.ReLU())
        self.conv7 = nn.Sequential(nn.Conv2d(1,3,kernel_size=(7,20),stride=1),nn.ReLU())
        self.conv8 = nn.Sequential(nn.Conv2d(1,3,kernel_size=(8,20),stride=1),nn.ReLU())
        self.linear = nn.Linear(21,output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input):
        input = self.encoder(input.long())
        print(input[0:5][:])
        input = input.squeeze(1)
#         print(input.unsqueeze(0).unsqueeze(0).size())
        output2 = self.conv2(input.unsqueeze(0).unsqueeze(0))
        self.maxpool = nn.MaxPool1d(kernel_size=(self.seq_length-2+1))
        output2 = self.maxpool(output2.squeeze(3))
#         print(output2.size())
        output3 = self.conv3(input.unsqueeze(0).unsqueeze(0))
        self.maxpool = nn.MaxPool1d(kernel_size=(self.seq_length-3+1))
        output3 = self.maxpool(output3.squeeze(3))
#         print(output3.size())
        output4 = self.conv4(input.unsqueeze(0).unsqueeze(0))
        self.maxpool = nn.MaxPool1d(kernel_size=(self.seq_length-4+1))
        output4 = self.maxpool(output4.squeeze(3))
#         print(output4.size())
        output5 = self.conv5(input.unsqueeze(0).unsqueeze(0))
        self.maxpool = nn.MaxPool1d(kernel_size=(self.seq_length-5+1))
        output5 = self.maxpool(output5.squeeze(3))
        
        output6 = self.conv6(input.unsqueeze(0).unsqueeze(0))
        self.maxpool = nn.MaxPool1d(kernel_size=(self.seq_length-6+1))
        output6 = self.maxpool(output6.squeeze(3))
        
        output7 = self.conv7(input.unsqueeze(0).unsqueeze(0))
        self.maxpool = nn.MaxPool1d(kernel_size=(self.seq_length-7+1))
        output7 = self.maxpool(output7.squeeze(3))
        
        output8 = self.conv8(input.unsqueeze(0).unsqueeze(0))
        self.maxpool = nn.MaxPool1d(kernel_size=(self.seq_length-8+1))
        output8 = self.maxpool(output8.squeeze(3))
        
        final_output = torch.cat((output2,output3,output4,output5,output6,output7,output8),1)
        final_output = self.linear(final_output.squeeze(2))
        final_output = self.softmax(final_output)
        return final_output

n_hidden = 64
cnn = CNN(seq_length,n_letters, n_hidden, n_categories)
    

In [5]:
input = lineToTensor(narr)
print(input.size())
hidden = torch.zeros(1,1,n_hidden)

output= cnn(input)
print(output)
print(output.size())


torch.Size([2077, 1])
tensor([[-4.1786, -2.6342, -3.0265, -3.2776, -4.4835, -5.6545, -3.1578,
         -3.1417, -2.9797, -3.4370, -3.5813, -1.4827, -3.6311, -1.8582,
         -2.4728, -2.7933, -4.1556, -3.1091]])
torch.Size([1, 18])


In [6]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
#     print(top_n)
#     print(top_i.size())
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

('10', 11)


In [7]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(data[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

def getTensors(category,line):
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor
    

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category,category_tensor, '/ line =', line)

category = 16 tensor([ 6]) / line = उत्तरदायी के अनुसार 45 वर्षीय पुरुष आत्महत्या के कारण मर गया था। वह स्वस्थ था। उसके पास भुगतान करने के लिए ऋण था जिसके कारण वह तनावपूर्ण रहता था। उसकी फसल भी खराब हो गई जिसके बाद उसने शराब का सेवन शुरू कर दिया। लगभग 2years से लगभग दैनिक शराब का उपभोग कर रहा था। एक दिन वह बहुत तनाव और शराब पीता था। कुछ समय बाद वह अपने खेतों में गया और कीटनाशक स्प्रे में प्रवेश किया और मौके पर मर गया। मुख्य रूप से वह आत्महत्या के कारण मर गया। कोई और समस्या नहीं थी।
category = 6 tensor([ 17]) / line = भुखमरी के कारण मृतक की मृत्यु के बाद मृतक का कोई शरीर नहीं था।
category = 1 tensor([ 7]) / line = उसने भूख खो दी और खुजली की शिकायत की जिसने उसे बेचैन बना दिया। उसने अपने शरीर में दर्द महसूस किया और ऊपरी और निचले पेट में अधिक गंभीर था। दिन में 5-6 बार नींद और सिरदर्द महसूस करना एक आम विशेषता थी या उसकी बीमारी के दौरान शिकायत थी। डॉक्टर से परामर्श किया गया था और सभी आवश्यक परीक्षणों और जांचों से परामर्श लिया गया था। कुछ दवाएं निर्धारित की गई थीं लेकिन केवल पेट दर्द को आंशिक

In [8]:
learning_rate = 0.005
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(cnn.parameters(),lr=learning_rate)

In [9]:
 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
#     hidden = rnn.initHidden()

    cnn.zero_grad()
    output = cnn(line_tensor)

    loss = criterion(output, category_tensor)
    loss.backward()
    optimizer.step()

    return output, loss.item()

def save():
    save_filename = "./model_cnn.pt"
    torch.save(cnn, save_filename)
    print('Saved as %s' % save_filename)
    
def writeToFile(line):
    if os.path.exists("output.txt"):
            append_write = 'a' # append if already exists
    else:
            append_write = 'w' # make a new file if not

    f = open("output.txt",append_write)
    f.write(line + '\n')
    f.close()
    
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [10]:
def trainIters(epochs):
    print_every = 5000
    counter = 0
    current_loss = 0
    start = time.time()
    for e in range(epochs):
        for k,v in data.iteritems():
            for i in range(len(v)):
                counter += 1
                category, line, category_tensor, line_tensor = getTensors(k,v[i])
                output, loss = train(category_tensor, line_tensor)
                current_loss += loss

        # Print iter number, loss, name and guess
                if counter % print_every == 0:
                    guess, guess_i = categoryFromOutput(output)
                    correct = '✓' if guess == category else '✗ (%s)' % category
                    line = '%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), current_loss, line, guess, correct)
                    print(line)
                    writeToFile(line)
        
    save()
                
                
            
     
    

In [92]:
if __name__ == '__main__':
    
    n_iters = 100000
    print_every = 5000
    plot_every = 1000

    # Keep track of losses for plotting
    current_loss = 0
    all_losses = []
    start = time.time()

    for iter in range(1, n_iters + 1):
        category, line, category_tensor, line_tensor = randomTrainingExample()
        output, loss = train(category_tensor, line_tensor)
        current_loss += loss

        # Print iter number, loss, name and guess
        if iter % print_every == 0:
            guess, guess_i = categoryFromOutput(output)
            correct = '✓' if guess == category else '✗ (%s)' % category
            print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

        # Add current loss avg to list of losses
        if iter % plot_every == 0:
            all_losses.append(current_loss / plot_every)
            current_loss = 0
    save()

tensor([[-2.1327]])
torch.Size([1, 1])
5000 5% (1m 14s) 2.9245  उत्तरदाता के अनुसार पूर्व मे उनको किसी प्रकार की बीमारी नही थी परंतु मृत्यु के  दिन पहले उनके दाहिने पैर मे दरद हुा दरद पांव की पिंडली से कमर तक होता था दवीयां भी दिया, आराम नही मिला, अधिक उमर हो गयी थी इसी कारण मृत्यु हो गयी / 15 ✗ (18)
tensor([[-2.1109]])
torch.Size([1, 1])
10000 10% (2m 27s) 2.9136  उत्तरदाता के अनुसार शाम करीब सात बजे का समय था उत्तरदाता घर के आअंगन से बढे थे मृतक बाहर से आये और चापाकल पर जाकर मुंह हांथ धोये उसके बाद अपने रूम मे जाने लगे लेकिन अचानक दरव पर ही वे गिर प सभी लोग मृतक को उठाकर देखा तो वे एकदम चित प हुये थे और उनका नबज़ चलना बंद हो गया था घरवाले पासके डौक्टर को बुलाये तो डौक्टर ने बताया कि वे अब इस दुनिया मे नही रहे / 15 ✗ (18)
tensor([[-2.0856]])
torch.Size([1, 1])
15000 15% (3m 39s) 2.7457  पहले से कोी बीमारी नहीं था उस रात को रम में बहुत दरद हुा दरद बरदास्त नहीं होता था ___ थोडा दरद ____ ____ ____ दिया गया लेकिन वो पेट में ____ हो उसकी मौत हो गयी / 15 ✗ (9)
tensor([[-2.0037]])
torch.Size

NameError: global name 'lstm' is not defined

In [13]:
evaluator = torch.load("./model_cnn.pt")
# -*- coding: UTF-8 -*-
test_data={}

tree = etree.parse('../data/mds+rct/dev_adult_cat.xml')
for e in tree.iter("cghr_cat"):
        if e.text not in test_data:
             test_data[e.text]=[]
for e in tree.iter("hindi_narrative","cghr_cat"):
    if e.tag == "hindi_narrative":
        value= u''.join(e.text)
#         print(value)
        
    if e.tag == 'cghr_cat':
        test_data[e.text].append(value)

total_test = 0
total_correct = 0
for k,v in test_data.items():
    cat = k
    for line in v:
        narr = line
        narr_tensor = lineToTensor(line)
        output = evaluator(narr_tensor)
        guess, guess_i = categoryFromOutput(output)
        print("correct cat: %s , predicted cat: %s"%(cat,guess))
        total_test +=1
        if cat == guess:
            total_correct +=1

print("total test: %d"%(total_test))
print("total correct: %d"%(total_correct))
print("prediction: %f"%((total_correct/total_test)*100))

tensor([[[ 0.2315, -0.1842, -0.7995,  3.0406,  1.2954,  0.9475, -0.1955,
          -0.0713,  0.2575,  1.2045, -0.1944,  0.9549,  0.9830,  0.4277,
           0.1594,  0.4076, -0.3752,  0.1504, -0.8346, -0.5927]],

        [[-0.3589,  0.3476, -0.6631,  0.4533,  0.2415,  1.6412,  0.9330,
           0.2302,  0.8732, -0.2929, -1.0948, -0.3090,  1.5746,  0.4253,
          -0.3840,  0.7291, -0.0996, -1.6558, -0.1545,  0.2722]],

        [[-0.1834,  0.1952, -0.2658,  0.4670,  0.0673,  0.5278, -0.0411,
          -0.3386, -0.2281,  0.0336,  0.8633,  0.2064,  0.6774,  0.4234,
           0.5758, -0.4875, -0.2876, -0.2961, -0.3364, -0.1636]],

        [[-0.4810,  0.1780,  0.2119,  1.2682, -0.0263,  0.1657, -0.5390,
           0.4519, -0.0992, -0.5111, -0.2907,  0.7772,  0.4490, -0.2201,
          -1.1858,  0.4830,  0.2096,  0.0169,  0.3398, -0.6345]],

        [[-0.1834,  0.1952, -0.2658,  0.4670,  0.0673,  0.5278, -0.0411,
          -0.3386, -0.2281,  0.0336,  0.8633,  0.2064,  0.6774,  0.4234,
  

correct cat: 10 , predicted cat: 8
tensor([[[ 0.2315, -0.1842, -0.7995,  3.0406,  1.2954,  0.9475, -0.1955,
          -0.0713,  0.2575,  1.2045, -0.1944,  0.9549,  0.9830,  0.4277,
           0.1594,  0.4076, -0.3752,  0.1504, -0.8346, -0.5927]],

        [[ 0.5530,  0.1299, -0.3754,  0.9753, -0.0975,  1.5880, -0.3790,
           0.4914,  0.0440,  0.6228, -0.0320,  0.6798,  2.1919,  0.9083,
           0.5339, -0.5060, -0.6649, -0.4904, -2.4192, -0.8364]],

        [[-1.0693,  1.2131, -1.3503,  2.1510, -0.6726, -0.0826,  0.2273,
           1.1122, -0.3005,  0.2689, -0.7169, -0.4311, -0.6265,  0.1909,
          -0.4027, -0.6084,  1.3884,  0.0268,  0.3514,  0.3063]],

        [[ 0.5063,  0.8354, -2.2708,  0.0510,  1.0622,  0.1393,  0.6286,
           0.6689,  0.7997, -0.0284, -0.9279, -0.1520,  0.6420,  0.1038,
           0.9761,  0.1944, -0.8463,  0.7925, -0.1407, -0.7476]],

        [[-1.0693,  1.2131, -1.3503,  2.1510, -0.6726, -0.0826,  0.2273,
           1.1122, -0.3005,  0.2689, -0.

tensor([[[ 0.2315, -0.1842, -0.7995,  3.0406,  1.2954,  0.9475, -0.1955,
          -0.0713,  0.2575,  1.2045, -0.1944,  0.9549,  0.9830,  0.4277,
           0.1594,  0.4076, -0.3752,  0.1504, -0.8346, -0.5927]],

        [[-0.2550,  0.4230, -0.9000,  0.4190,  0.1317, -0.3000,  0.0915,
           0.0121,  0.1092,  1.0798,  0.2754,  0.6002,  0.7409,  0.4744,
           0.2733, -0.1159, -0.0513, -0.3320,  0.4193, -1.1951]],

        [[ 0.8458,  0.9523, -1.8674,  0.5357,  0.9400,  1.5246,  0.7122,
           0.9118,  0.2687,  1.7787, -0.5508,  1.0936,  2.2947, -0.1584,
           1.1612, -0.6460, -0.7102,  0.9873,  0.4262, -0.1639]],

        [[-0.1834,  0.1952, -0.2658,  0.4670,  0.0673,  0.5278, -0.0411,
          -0.3386, -0.2281,  0.0336,  0.8633,  0.2064,  0.6774,  0.4234,
           0.5758, -0.4875, -0.2876, -0.2961, -0.3364, -0.1636]],

        [[ 0.7525, -0.7296, -0.6614,  1.3898,  0.5602,  1.2274, -0.0502,
           0.0593,  0.2114,  0.2061,  0.3363,  0.4308,  1.0309,  0.5831,
  

tensor([[[ 0.2315, -0.1842, -0.7995,  3.0406,  1.2954,  0.9475, -0.1955,
          -0.0713,  0.2575,  1.2045, -0.1944,  0.9549,  0.9830,  0.4277,
           0.1594,  0.4076, -0.3752,  0.1504, -0.8346, -0.5927]],

        [[-0.3589,  0.3476, -0.6631,  0.4533,  0.2415,  1.6412,  0.9330,
           0.2302,  0.8732, -0.2929, -1.0948, -0.3090,  1.5746,  0.4253,
          -0.3840,  0.7291, -0.0996, -1.6558, -0.1545,  0.2722]],

        [[-0.1834,  0.1952, -0.2658,  0.4670,  0.0673,  0.5278, -0.0411,
          -0.3386, -0.2281,  0.0336,  0.8633,  0.2064,  0.6774,  0.4234,
           0.5758, -0.4875, -0.2876, -0.2961, -0.3364, -0.1636]],

        [[-0.4810,  0.1780,  0.2119,  1.2682, -0.0263,  0.1657, -0.5390,
           0.4519, -0.0992, -0.5111, -0.2907,  0.7772,  0.4490, -0.2201,
          -1.1858,  0.4830,  0.2096,  0.0169,  0.3398, -0.6345]],

        [[-0.1834,  0.1952, -0.2658,  0.4670,  0.0673,  0.5278, -0.0411,
          -0.3386, -0.2281,  0.0336,  0.8633,  0.2064,  0.6774,  0.4234,
  

tensor([[[ 0.2315, -0.1842, -0.7995,  3.0406,  1.2954,  0.9475, -0.1955,
          -0.0713,  0.2575,  1.2045, -0.1944,  0.9549,  0.9830,  0.4277,
           0.1594,  0.4076, -0.3752,  0.1504, -0.8346, -0.5927]],

        [[-0.3589,  0.3476, -0.6631,  0.4533,  0.2415,  1.6412,  0.9330,
           0.2302,  0.8732, -0.2929, -1.0948, -0.3090,  1.5746,  0.4253,
          -0.3840,  0.7291, -0.0996, -1.6558, -0.1545,  0.2722]],

        [[-0.1834,  0.1952, -0.2658,  0.4670,  0.0673,  0.5278, -0.0411,
          -0.3386, -0.2281,  0.0336,  0.8633,  0.2064,  0.6774,  0.4234,
           0.5758, -0.4875, -0.2876, -0.2961, -0.3364, -0.1636]],

        [[-0.4810,  0.1780,  0.2119,  1.2682, -0.0263,  0.1657, -0.5390,
           0.4519, -0.0992, -0.5111, -0.2907,  0.7772,  0.4490, -0.2201,
          -1.1858,  0.4830,  0.2096,  0.0169,  0.3398, -0.6345]],

        [[-0.1834,  0.1952, -0.2658,  0.4670,  0.0673,  0.5278, -0.0411,
          -0.3386, -0.2281,  0.0336,  0.8633,  0.2064,  0.6774,  0.4234,
  

...

correct cat: 7 , predicted cat: 8
tensor([[[ 0.2315, -0.1842, -0.7995,  3.0406,  1.2954,  0.9475, -0.1955,
          -0.0713,  0.2575,  1.2045, -0.1944,  0.9549,  0.9830,  0.4277,
           0.1594,  0.4076, -0.3752,  0.1504, -0.8346, -0.5927]],

        [[-0.3589,  0.3476, -0.6631,  0.4533,  0.2415,  1.6412,  0.9330,
           0.2302,  0.8732, -0.2929, -1.0948, -0.3090,  1.5746,  0.4253,
          -0.3840,  0.7291, -0.0996, -1.6558, -0.1545,  0.2722]],

        [[-0.1834,  0.1952, -0.2658,  0.4670,  0.0673,  0.5278, -0.0411,
          -0.3386, -0.2281,  0.0336,  0.8633,  0.2064,  0.6774,  0.4234,
           0.5758, -0.4875, -0.2876, -0.2961, -0.3364, -0.1636]],

        [[-0.4810,  0.1780,  0.2119,  1.2682, -0.0263,  0.1657, -0.5390,
           0.4519, -0.0992, -0.5111, -0.2907,  0.7772,  0.4490, -0.2201,
          -1.1858,  0.4830,  0.2096,  0.0169,  0.3398, -0.6345]],

        [[-0.1834,  0.1952, -0.2658,  0.4670,  0.0673,  0.5278, -0.0411,
          -0.3386, -0.2281,  0.0336,  0.8

tensor([[[ 0.2315, -0.1842, -0.7995,  3.0406,  1.2954,  0.9475, -0.1955,
          -0.0713,  0.2575,  1.2045, -0.1944,  0.9549,  0.9830,  0.4277,
           0.1594,  0.4076, -0.3752,  0.1504, -0.8346, -0.5927]],

        [[-0.3589,  0.3476, -0.6631,  0.4533,  0.2415,  1.6412,  0.9330,
           0.2302,  0.8732, -0.2929, -1.0948, -0.3090,  1.5746,  0.4253,
          -0.3840,  0.7291, -0.0996, -1.6558, -0.1545,  0.2722]],

        [[-0.1834,  0.1952, -0.2658,  0.4670,  0.0673,  0.5278, -0.0411,
          -0.3386, -0.2281,  0.0336,  0.8633,  0.2064,  0.6774,  0.4234,
           0.5758, -0.4875, -0.2876, -0.2961, -0.3364, -0.1636]],

        [[-0.4810,  0.1780,  0.2119,  1.2682, -0.0263,  0.1657, -0.5390,
           0.4519, -0.0992, -0.5111, -0.2907,  0.7772,  0.4490, -0.2201,
          -1.1858,  0.4830,  0.2096,  0.0169,  0.3398, -0.6345]],

        [[-0.1834,  0.1952, -0.2658,  0.4670,  0.0673,  0.5278, -0.0411,
          -0.3386, -0.2281,  0.0336,  0.8633,  0.2064,  0.6774,  0.4234,
  

tensor([[[ 0.2315, -0.1842, -0.7995,  3.0406,  1.2954,  0.9475, -0.1955,
          -0.0713,  0.2575,  1.2045, -0.1944,  0.9549,  0.9830,  0.4277,
           0.1594,  0.4076, -0.3752,  0.1504, -0.8346, -0.5927]],

        [[-0.3589,  0.3476, -0.6631,  0.4533,  0.2415,  1.6412,  0.9330,
           0.2302,  0.8732, -0.2929, -1.0948, -0.3090,  1.5746,  0.4253,
          -0.3840,  0.7291, -0.0996, -1.6558, -0.1545,  0.2722]],

        [[-0.0549, -0.3718,  0.3888,  1.7781, -0.4985,  0.9459,  1.0095,
           0.1888,  0.2231, -0.0640,  0.5193,  0.1436,  0.5281, -0.1972,
          -0.0303,  0.2773,  0.2590, -0.6293,  0.1411, -0.3071]],

        [[ 0.7525, -0.7296, -0.6614,  1.3898,  0.5602,  1.2274, -0.0502,
           0.0593,  0.2114,  0.2061,  0.3363,  0.4308,  1.0309,  0.5831,
          -0.3559, -0.1647,  0.3922,  0.2091, -0.5601, -1.1288]],

        [[ 0.5891, -0.5307, -0.4034,  1.0289,  0.5505,  0.6708, -0.2484,
           0.6381, -0.1436,  0.0176,  0.5319,  0.4765,  0.8624, -0.1436,
  

correct cat: 8 , predicted cat: 8
tensor([[[ 0.2315, -0.1842, -0.7995,  3.0406,  1.2954,  0.9475, -0.1955,
          -0.0713,  0.2575,  1.2045, -0.1944,  0.9549,  0.9830,  0.4277,
           0.1594,  0.4076, -0.3752,  0.1504, -0.8346, -0.5927]],

        [[-0.2550,  0.4230, -0.9000,  0.4190,  0.1317, -0.3000,  0.0915,
           0.0121,  0.1092,  1.0798,  0.2754,  0.6002,  0.7409,  0.4744,
           0.2733, -0.1159, -0.0513, -0.3320,  0.4193, -1.1951]],

        [[ 0.8458,  0.9523, -1.8674,  0.5357,  0.9400,  1.5246,  0.7122,
           0.9118,  0.2687,  1.7787, -0.5508,  1.0936,  2.2947, -0.1584,
           1.1612, -0.6460, -0.7102,  0.9873,  0.4262, -0.1639]],

        [[-0.1834,  0.1952, -0.2658,  0.4670,  0.0673,  0.5278, -0.0411,
          -0.3386, -0.2281,  0.0336,  0.8633,  0.2064,  0.6774,  0.4234,
           0.5758, -0.4875, -0.2876, -0.2961, -0.3364, -0.1636]],

        [[ 0.7525, -0.7296, -0.6614,  1.3898,  0.5602,  1.2274, -0.0502,
           0.0593,  0.2114,  0.2061,  0.3

tensor([[[ 0.2315, -0.1842, -0.7995,  3.0406,  1.2954,  0.9475, -0.1955,
          -0.0713,  0.2575,  1.2045, -0.1944,  0.9549,  0.9830,  0.4277,
           0.1594,  0.4076, -0.3752,  0.1504, -0.8346, -0.5927]],

        [[-0.3589,  0.3476, -0.6631,  0.4533,  0.2415,  1.6412,  0.9330,
           0.2302,  0.8732, -0.2929, -1.0948, -0.3090,  1.5746,  0.4253,
          -0.3840,  0.7291, -0.0996, -1.6558, -0.1545,  0.2722]],

        [[-0.1834,  0.1952, -0.2658,  0.4670,  0.0673,  0.5278, -0.0411,
          -0.3386, -0.2281,  0.0336,  0.8633,  0.2064,  0.6774,  0.4234,
           0.5758, -0.4875, -0.2876, -0.2961, -0.3364, -0.1636]],

        [[-0.4810,  0.1780,  0.2119,  1.2682, -0.0263,  0.1657, -0.5390,
           0.4519, -0.0992, -0.5111, -0.2907,  0.7772,  0.4490, -0.2201,
          -1.1858,  0.4830,  0.2096,  0.0169,  0.3398, -0.6345]],

        [[-0.1834,  0.1952, -0.2658,  0.4670,  0.0673,  0.5278, -0.0411,
          -0.3386, -0.2281,  0.0336,  0.8633,  0.2064,  0.6774,  0.4234,
  